In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from torch.nn import BCEWithLogitsLoss, BCELoss
from tqdm import tqdm_notebook

from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important;}</style>"))
pd.set_option('display.max_rows', -1)
pd.set_option('display.max_columns', 100)

%run ../chicm/model.py
%run ../chicm/loader.py
%run ../lib/checks.py
%run ../lib/train_class_edition.py
%run ../lib/common_.py
%run ../lib/optim.py
%run ../lib/read_pandas.py

DEVICE = torch.device('cuda')

### Заметки

### Train part

In [2]:
train = pd.read_csv('../data/train.csv')
submission = pd.read_csv('../data/sample_submission.csv')
print(train.shape)
target_columns = submission.columns.values[1:].tolist()     

(6079, 41)


In [3]:
model_params = {
    'dropout': .2,
    'tokenizer': 'bert-base-cased',
    'lr': 3e-5
}

kfold_params = {
    'random_state': 66,
    'split_ratio': .8
}

In [4]:
train_loader, val_loader = get_train_val_loaders(train, 
                                                 target_columns,
                                                 random_state=kfold_params['random_state'],
                                                 split_ratio=kfold_params['split_ratio'],
                                                 tokenizer=model_params['tokenizer'])
model = QuestModel(n_classes=30,
                   dropout=model_params['dropout'],
                   tokenizer=model_params['tokenizer']).cuda()

loss = BCEWithLogitsLoss().cuda()
optimizer = configure_optimizers(model, model_params['lr'])

1216


In [5]:
# def create_model(model_file):
#     model = QuestModel()
#     model.load_state_dict(torch.load(model_file))
#     model = model.cuda()
#     return model   
# model = create_model('../data/result/01-23-12/models/bert_model_epoch9')

In [6]:
trainer = BertPredictor(model,
                        train_loader,
                        loss, 
                        optimizer,
                        val_loader=val_loader,
                        split_rand_state=kfold_params['random_state'],
                        epochs_count=8)

In [7]:
trainer.fit()

[1 / 8] Train: Loss = 0.41626:  58%|█████▊    | 700/1215 [03:44<02:52,  2.98it/s]

0.16789390283780048
val_score --> 0.25314639640629166


  0%|          | 0/1215 [00:00<?, ?it/s]

val_score --> 0.2905770620040772


[2 / 8] Train: Loss = 0.36092:  58%|█████▊    | 700/1215 [03:38<02:40,  3.20it/s]

...

0.558413434649476


[8 / 8] Train: Loss = 0.22551:  58%|█████▊    | 701/1215 [04:09<1:37:46, 11.41s/it]

val_score --> 0.3656371417972661


[8 / 8] Train: Loss = 0.30621: 100%|██████████| 1215/1215 [06:45<00:00,  2.99it/s] 


val_score --> 0.36819626566122515


### Prediction part

In [ ]:
def create_model(model_file):
    model = QuestModel()
    model.load_state_dict(torch.load(model_file))
    model = model.cuda()
    return model


def predict_test(test_loader, new_model):
    test_preds_lst = []
    with torch.no_grad():
        for batch_id, batch_seg in tqdm(test_loader):
            logits = new_model(batch_id.cuda(), batch_seg.cuda())
            test_preds_lst.append(to_numpy(logits)) 
    return test_preds_lst


def _get_target_cols():
    submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))
    target_columns = submission.columns.values[1:].tolist()  
    return target_columns

In [ ]:
target_columns = _get_target_cols()
test_preds_lst = []
test_loader = get_test_loader(target_cols=target_columns)
for i_model in tqdm_notebook(range(2, 6)):
    model = create_model(f'../data/result/models/bert_model_epoch{i_model}') 
    test_preds = predict_test(test_loader, model)
    test_preds_lst.append(np.concatenate(test_preds))

In [ ]:
sub1 = pd.read_csv('../data/result/submission/submission.csv')
sub2 = pd.read_csv('../data/result/submission/submission2.csv')
sub3 = pd.read_csv('../data/result/submission/submission_21_12.csv')

In [ ]:
(sub1.mean(0) - sub2.mean(0)).head(2)

In [ ]:
(sub1.mean(0) - sub3.mean(0)).head(2)